## Question 1: **Answer**

In [1]:
import sqlite3
import csv

In [ ]:
def load_student_data(input_data):
    try:
        # Connecting to database and creating cursor
        conn = sqlite3.connect("student_marks_record.db")
        cursor = conn.cursor()
        # Creating table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS results (
                ID PRIMARY KEY,
                first_name TEXT,
                last_name TEXT,
                marks INT
            )
        """
                )
        with open(input_data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # Skip the header row
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO results VALUES (?, ?, ?, ?)", row)
        print("Data successfully saved in the database")
        conn.commit()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

# Loading data
data = "/content/student_marks.csv"
load_student_data(data)

Error: UNIQUE constraint failed: results.ID


In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///student_marks_record.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM results
LIMIT 5;

 * sqlite:///student_marks_record.db
Done.


ID,first_name,last_name,marks
101,Emma,Smith,85
102,Liam,Johnson,92
103,Olivia,Williams,78
104,Noah,Brown,95
105,Ava,Jones,88


## Question 2: **Answer**

In [7]:
%%sql
SELECT
    COUNT(*) AS marks_between_80_and_90
FROM results
WHERE marks BETWEEN 80 AND 90;

 * sqlite:///student_marks_record.db
Done.


marks_between_80_and_90
14


## Question 3: **Answer**

In [8]:
%%sql
SELECT
    last_name,
    marks
FROM results
ORDER BY marks DESC
LIMIT 1;

 * sqlite:///student_marks_record.db
Done.


last_name,marks
Taylor,96


## Question 4: **Answer**

In [9]:
%%sql
SELECT
    first_name,
    last_name,
    marks
FROM results
WHERE first_name IN ('Abigail', 'Caden',
                  'Ella', 'Logan', 'Mia')
ORDER BY marks DESC;

 * sqlite:///student_marks_record.db
Done.


first_name,last_name,marks
Abigail,Thomas,95
Mia,Garcia,94
Ella,Brown,89
Logan,Anderson,78
Caden,Smith,77


## Question 5: **Answer**

In [10]:
%%sql
SELECT
    marks,
    COUNT(*) AS num_of_students
FROM results
GROUP BY marks
ORDER BY num_of_students DESC
LIMIT 5;

 * sqlite:///student_marks_record.db
Done.


marks,num_of_students
89,3
95,2
94,2
92,2
91,2


## Question 6: **Answer**

In [11]:
%%sql
SELECT
    AVG(marks) AS average_marks_abigail_amelia
FROM results
WHERE first_name IN ('Abigail', 'Amelia');

 * sqlite:///student_marks_record.db
Done.


average_marks_abigail_amelia
92.5


## Question 7: **Answer**

In [12]:
%%sql
SELECT * FROM
     --subquery to return student with lowest mark
            (SELECT
                first_name,
                last_name,
                marks
            FROM results
            ORDER BY marks ASC
            LIMIT 1)
            AS student_lowest_mark,
    --subquery to return student with highest mark
            (SELECT
                first_name,
                last_name,
                marks
            FROM results
            ORDER BY marks DESC
            LIMIT 1)
            AS student_highest_mark;

 * sqlite:///student_marks_record.db
Done.


first_name,last_name,marks,first_name_1,last_name_1,marks_1
Lucas,Davis,76,Zoe,Taylor,96


## Question 8: **Answer**

In [13]:
%%sql
-- Ranking students based on marks
WITH ranked_results AS (
    SELECT
        first_name,
        last_name,
        marks,
        DENSE_RANK() OVER (ORDER BY marks ASC) AS position
    FROM results
)
-- Select students ranked 5th or lower
SELECT
    first_name,
    last_name,
    marks,
    position
FROM ranked_results
WHERE position <= 5;

 * sqlite:///student_marks_record.db
Done.


first_name,last_name,marks,position
Lucas,Davis,76,1
Caden,Smith,77,2
Olivia,Williams,78,3
Logan,Anderson,78,3
Aiden,Smith,79,4
Ethan,Smith,79,4
Aiden,Martinez,80,5


In [14]:
# %%sql
# DROP TABLE results;